# Append 2017-2018 data

* It doesn't come in HTML so the R script cannot be applied
* It doesn't have 12-17 age group

In [185]:
import pandas as pd
import numpy as np
import csv

In [170]:
# Existing file with data up to 2017
base = pd.read_csv('../data/dmhas_nsduh_mh_2017.csv')

# Excel file with the new data
xls = pd.ExcelFile('../raw/NSDUHsaeExcelTabs2018.xlsx')

In [171]:
# Geo crosswalk
clean_geo = {
    'Total U.S.': 'United States',
    'Midwest': 'Midwestern Region',
    'Northeast': 'Northeastern Region',
    'South': 'Southern Region',
    'West': 'Western Region'
}

In [172]:
def clean_up_table(df):
    
    # Only relevant geographies
    df = df[ df.State.isin(['Total U.S.', 'Northeast', 'Midwest', 'South', 'West', 'Connecticut']) ]
    df.State = df.State.apply(lambda x: clean_geo[x] if x in clean_geo else x)
    
    # Clean up column names
    df.drop( list(df.filter(regex='CI')), axis=1, inplace=True )
    
    # Clean up column names
    df.columns = [ x.split('\n')[0] for x in df.columns.values ]
    df = df.rename(columns={'18 or Older': 'Over 17', '26 or Older': 'Over 25', 'State': 'Region'})
    
    # Express as percentages
    df.iloc[:,1:] = np.round(df.iloc[:,1:] * 100, 2)
    
    # Add 12-17 age group
    df['12-17'] = -6666
    
    return df

### Serious Mental Illness

In [173]:
# Make sure Table 27 is indeed Serious Mental Illness
assert 'Serious Mental Illness' in pd.read_excel(xls, 'Table 27').columns[0]

mn_serious = pd.read_excel(xls, 'Table 27', skiprows=5, index_col='Order')
mn_serious = clean_up_table(mn_serious).melt(id_vars='Region', var_name='Age Range', value_name='Value')
mn_serious['Mental Health'] = 'Serious Mental Illness in the Past Year'

### Any Mental Illness

In [174]:
# Make sure Table 28 is indeed Any Mental Illness
assert 'Any Mental Illness' in pd.read_excel(xls, 'Table 28').columns[0]

mn_any = pd.read_excel(xls, 'Table 28', skiprows=5, index_col='Order')
mn_any = clean_up_table(mn_any).melt(id_vars='Region', var_name='Age Range', value_name='Value')
mn_any['Mental Health'] = 'Any Mental Illness in the Past Year'

### Thoughts of Suicide

In [175]:
# Make sure Table 30 is indeed Serious Thoughts of Suicide
assert 'Serious Thoughts of Suicide' in pd.read_excel(xls, 'Table 30').columns[0]

mn_suicide = pd.read_excel(xls, 'Table 30', skiprows=5, index_col='Order')
mn_suicide = clean_up_table(mn_suicide).melt(id_vars='Region', var_name='Age Range', value_name='Value')
mn_suicide['Mental Health'] = 'Had Serious Thoughts of Suicide in the Past Year'

### Major Depressive Episode

In [176]:
# Make sure Table 30 is indeed Major Depressive Episode
assert 'Major Depressive Episode' in pd.read_excel(xls, 'Table 31').columns[0]

mn_depress = pd.read_excel(xls, 'Table 31', skiprows=6, index_col='Order')
mn_depress = clean_up_table(mn_depress).melt(id_vars='Region', var_name='Age Range', value_name='Value')
mn_depress['Mental Health'] = 'Had at Least One Major Depressive Episode in the Past Year'

## Combine all into one and save to base

In [187]:
new_data = pd.concat([
    mn_serious,
    mn_any,
    mn_suicide,
    mn_depress
])

new_data['Year'] = '2017-2018'
new_data['Variable'] = 'Mental Health'
new_data['Measure Type'] = 'Percent'

base.append(new_data).sort_values(['Region', 'Year', 'Age Range']).filter([
    'Region',
    'Year',
    'Age Range',
    'Mental Health',
    'Measure Type',
    'Variable',
    'Value'
]).to_csv('../data/dmhas_nsduh_mn_2018.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)